In [1]:
import pandas as pd
import numpy as np
import h5py
import sys
sys.path.append("..")
from os import listdir
from os.path import isfile, join
import random
random.seed(42)
from scipy.spatial import KDTree

In [2]:
data_dir = "../data/labeled/"
h5_small_full = "../data/h5_meter/data.h5py"
h5_small_train = "../data/h5_meter/train.h5py"
h5_small_test = "../data/h5_meter/test.h5py"
h5_small_validate = "../data/h5_meter/validate.h5py"

In [3]:
# Final tile size in meters
GRID_TILE_SIZE = 1

In [4]:
files = [f for f in listdir(data_dir) if isfile(join(data_dir, f))]

In [29]:
# Splits all the tiles into GRID_TILE_SIZE
count = 0
with h5py.File(h5_small_full, 'w') as h5f:
    for file in files:
        df = pd.read_csv(join(data_dir, file), sep=" ", header=None)
        data = df.to_numpy()
        xyz_max = np.array([np.max(data[:,0]), np.max(data[:,1]), np.max(data[:,2])])
        xyz_min = np.array([np.min(data[:,0]), np.min(data[:,1]), np.min(data[:,2])])
        x_min, y_min, z_min = xyz_min
        xyz_len = xyz_max - xyz_min
        xyz_grid_num = np.ceil(xyz_len / GRID_TILE_SIZE)
        print("Processing Tile:",  file)
        print('Grid Shape: ', xyz_grid_num)
        tile_keys = []
        for x in range(int(xyz_grid_num[0])):
            bool_x_array = np.logical_and(data[:,0] <= (x_min + GRID_TILE_SIZE * (x + 1)), data[:,0] >= (x_min + GRID_TILE_SIZE * x))
            for y in range(int(xyz_grid_num[1])):
                bool_y_array = np.logical_and(data[:,1] <= (y_min + GRID_TILE_SIZE * (y + 1)), data[:,1] >= (y_min + GRID_TILE_SIZE * y))
                for z in range(int(xyz_grid_num[2])):
                    bool_z_array = np.logical_and(data[:,2] <= (z_min + GRID_TILE_SIZE * (z + 1)), data[:,2] >= (z_min + GRID_TILE_SIZE * z))
                    tile_data = data[np.logical_and(bool_x_array, np.logical_and(bool_y_array, bool_z_array))]
                    if tile_data.shape[0] == 0: # if tile contains zero points skip it
                        print("Skipping: tile contains 0 points.")
                        continue
                    coords = tile_data[:,0:3]
                    feats = tile_data[:,3:5]
                    labels = tile_data[:,-1]
                    tile_keys.append("tile_" + str(count))
                    tile = h5f.create_group("tile_" + str(count))
                    tile.create_dataset('coords', data = coords)
                    tile.create_dataset("feats", data = feats)
                    tile.create_dataset('labels', data = labels)
                    print("Number of Mini Tiles",  count)
                    count += 1
        deleted_keys = []
        for key in tile_keys:
            if h5f[key]["coords"].shape[0] < 5:
                print("Small:", key)
                coords = h5f[key]["coords"][:]
                feats = h5f[key]["feats"][:]
                labels = h5f[key]["labels"][:]
                xyz_max = np.array([np.max(coords[:,0]), np.max(coords[:,1]), np.max(coords[:,2])])
                xyz_min = np.array([np.min(coords[:,0]), np.min(coords[:,1]), np.min(coords[:,2])])
                print(xyz_max)
                print(xyz_min)
                closest_tile = None
                closest_max = None 
                closest_min = None 
                for key_2 in tile_keys:
                    if key_2 == key or key_2 in deleted_keys:
                        continue
                    current_tile_data = h5f[key_2]["coords"][:]
                    close_max = current_tile_data[KDTree(current_tile_data).query(xyz_max)[1]]
                    close_min = current_tile_data[KDTree(current_tile_data).query(xyz_min)[1]]
                    if closest_tile is None:
                        closest_tile = key_2
                        closest_max = close_max
                        closest_min = close_min
                    else:
                        compare_max_arr = [closest_max, close_max]
                        temp_closest_max = compare_max_arr[KDTree(compare_max_arr).query(xyz_max)[1]]
                        compare_min_arr = [closest_min, close_min]
                        temp_closest_min = compare_min_arr[KDTree(compare_min_arr).query(xyz_max)[1]]
                        if not np.array_equal(temp_closest_max, closest_max) and not np.array_equal(temp_closest_min, closest_min):
                            closest_tile = key_2
                            closest_max = temp_closest_max
                            closest_min = temp_closest_min
                print("Closest:", closest_tile)
                print(closest_max)
                print(closest_min)
                print("Merging:", key, "to", closest_tile, "\n")
                np.concatenate((h5f[closest_tile]["coords"][:], coords))
                np.concatenate((h5f[closest_tile]["feats"][:], feats))
                np.concatenate((h5f[closest_tile]["labels"][:], labels))
                del h5f[key]
                deleted_keys.append(key)
        print("Merged Tiles:", str(len(deleted_keys)))
        

Processing Tile: labeled_tile_-10_0.txt
Grid Shape:  [10. 10. 32.]
Number of Mini Tiles 0
Number of Mini Tiles 1
Number of Mini Tiles 2
Number of Mini Tiles 3
Number of Mini Tiles 4
Number of Mini Tiles 5
Number of Mini Tiles 6
Number of Mini Tiles 7
Number of Mini Tiles 8
Number of Mini Tiles 9
Number of Mini Tiles 10
Number of Mini Tiles 11
Number of Mini Tiles 12
Number of Mini Tiles 13
Number of Mini Tiles 14
Number of Mini Tiles 15
Number of Mini Tiles 16
Number of Mini Tiles 17
Number of Mini Tiles 18
Number of Mini Tiles 19
Number of Mini Tiles 20
Number of Mini Tiles 21
Number of Mini Tiles 22
Number of Mini Tiles 23
Number of Mini Tiles 24
Number of Mini Tiles 25
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Number of Mini Tiles 26
Number of Mini Tiles 27
Number of Mini Tiles 28
Number of Mini Tiles 29
Number of Mini Tile

Number of Mini Tiles 205
Number of Mini Tiles 206
Number of Mini Tiles 207
Number of Mini Tiles 208
Number of Mini Tiles 209
Number of Mini Tiles 210
Skipping: tile contains 0 points.
Number of Mini Tiles 211
Number of Mini Tiles 212
Number of Mini Tiles 213
Number of Mini Tiles 214
Number of Mini Tiles 215
Number of Mini Tiles 216
Number of Mini Tiles 217
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Number of Mini Tiles 218
Number of Mini Tiles 219
Number of Mini Tiles 220
Number of Mini Tiles 221
Number of Mini Tiles 222
Number of Mini Tiles 223
Number of Mini Tiles 224
Number of Mini Tiles 225
Number of Mini Tiles 226
Number of Mini Tiles 227
Number of Mini Tiles 228
Number of Mini Tiles 229
Number of Mini Tiles 230
Number of 

Number of Mini Tiles 411
Number of Mini Tiles 412
Number of Mini Tiles 413
Number of Mini Tiles 414
Number of Mini Tiles 415
Number of Mini Tiles 416
Number of Mini Tiles 417
Number of Mini Tiles 418
Number of Mini Tiles 419
Number of Mini Tiles 420
Number of Mini Tiles 421
Number of Mini Tiles 422
Number of Mini Tiles 423
Number of Mini Tiles 424
Number of Mini Tiles 425
Number of Mini Tiles 426
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Number of Mini Tiles 427
Number of Mini Tiles 428
Number of Mini Tiles 429
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Number of Mini Tiles 430
Number of Mini Tiles 431
Number of Mini Tiles 432
Number o

Skipping: tile contains 0 points.
Number of Mini Tiles 641
Number of Mini Tiles 642
Number of Mini Tiles 643
Number of Mini Tiles 644
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Number of Mini Tiles 645
Number of Mini Tiles 646
Number of Mini Tiles 647
Number of Mini Tiles 648
Number of Mini Tiles 649
Number of Mini Tiles 650
Number of Mini Tiles 651
Number of Mini Tiles 652
Number of Mini Tiles 653
Number of Mini Tiles 654
Number of Mini Tiles 655
Number of Mini Tiles 656
Number of Mini Tiles 657
Number of Mini Tiles 658
Number of Mini Tiles 659
Number of Mini Tiles 660
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Number of Mini Tiles 661
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.

Number of Mini Tiles 879
Number of Mini Tiles 880
Number of Mini Tiles 881
Number of Mini Tiles 882
Number of Mini Tiles 883
Number of Mini Tiles 884
Number of Mini Tiles 885
Skipping: tile contains 0 points.
Number of Mini Tiles 886
Number of Mini Tiles 887
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Number of Mini Tiles 888
Number of Mini Tiles 889
Number of Mini Tiles 890
Number of Mini Tiles 891
Number of Mini Tiles 892
Number of Mini Tiles 893
Number of Mini Tiles 894
Number of Mini Tiles 895
Number of Mini Tiles 896
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Number of Mini Tiles 897
Number of Mini Tiles 898
Number of Mini Tiles 899
Number of Mini Tiles 900
Number of Mini Tiles 901
Number of Mini Tiles 902
Number of Mini Tiles 903
Number of Mini Tiles 904
Skipping: tile contains 0 points.
Number of Mini Tiles 905
Number of Mini Tiles 906
Number of Mini Tiles 907
Number of Mini Tiles 908
Number of Mini Tiles 909
Number of Mini Tiles 

...

Number of Mini Tiles 55422
Number of Mini Tiles 55423
Number of Mini Tiles 55424
Number of Mini Tiles 55425
Number of Mini Tiles 55426
Number of Mini Tiles 55427
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Number of Mini Tiles 55428
Number of Mini Tiles 55429
Number of Mini Tiles 55430
Number of Mini Tiles 55431
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Number of Mini Tiles 55432
Number of Mini Tiles 55433
Number of Mini Tiles 55434
Number of Mini Tiles 55435
Number of Mini Tiles 55436
Number of Mini Tiles 55437
Number of Mini Tiles 55438
Number of Mini Tiles 55439
Skipping: tile contains 0 points.
Numb

Number of Mini Tiles 55634
Number of Mini Tiles 55635
Number of Mini Tiles 55636
Number of Mini Tiles 55637
Number of Mini Tiles 55638
Number of Mini Tiles 55639
Number of Mini Tiles 55640
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Number of Mini Tiles 55641
Number of Mini Tiles 55642
Number of Mini Tiles 55643
Number of Mini Tiles 55644
Number of Mini Tiles 55645
Number of Mini Tiles 55646
Number of Mini Tiles 55647
Number of Mini Tiles 55648
Number of Mini Tiles 55649
Number of Mini Tiles 55650
Number of Mini Tiles 55651
Number of Mini Tiles 55652
Number of Mini Tiles 55653
Number of Mini Tiles 55654
Number of Mini Tiles 55655
Number of Mini Tiles 55656
Number of Mini Tiles 55657
Number of Mini Tiles 55658
Number of Mini Tiles 55659
Number of Mini Tiles 55660
Number of Mini Tiles 55661
Number of Mini Tiles 55662
Number of Min

Number of Mini Tiles 55868
Number of Mini Tiles 55869
Number of Mini Tiles 55870
Number of Mini Tiles 55871
Number of Mini Tiles 55872
Number of Mini Tiles 55873
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Number of Mini Tiles 55874
Number of Mini Tiles 55875
Number of Mini Tiles 55876
Number of Mini Tiles 55877
Number of Mini Tiles 55878
Number of Mini Tiles 55879
Number of Mini Tiles 55880
Number of Mini Tiles 55881
Number of Mini Tiles 55882
Number of Mini Tiles 55883
Number of Mini Tiles 55884
Number of Mini Tiles 55885
Number of Mini Tiles 55886
Number of Mini Tiles 55887
Number of Mini Tiles 55888
Number of Mini Tiles 55889
Number of Mini Tiles 55890
Number of Mini Tiles 55891
Number of Mini Tiles 55892
Number of Mini Tiles 55893
Number of Mini Tiles 55894
Number of Mini Tiles 55895
Number of Mini Tiles 55896
Skipping: til

Number of Mini Tiles 56096
Number of Mini Tiles 56097
Number of Mini Tiles 56098
Number of Mini Tiles 56099
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Number of Mini Tiles 56100
Number of Mini Tiles 56101
Number of Mini Tiles 56102
Number of Mini Tiles 56103
Number of Mini Tiles 56104
Number of Mini Tiles 56105
Number of Mini Tiles 56106
Number of Mini Tiles 56107
Number of Mini Tiles 56108
Number of Mini Tiles 56109
Number of Mini Tiles 56110
Number of Mini Tiles 56111
Number of Mini Tiles 56112
Number of Mini Tiles 56113
Number of Mini Tiles 56114
Number of Mini Tiles 56115
Number of Mini Tiles 56116
Number of Mini Tiles 56117
Skipping: tile contains 0 points.
Number of Mini Tiles 56118
Number of Mini Tiles 56119
Number of Mini Tiles 56120
Number of Mini Tiles 56121
Skipping: tile contains 0 points.
Skipping: tile contains 0 points.
Skipping: tile contains 0 

Closest: tile_54795
[-36.84375  -8.986   160.955  ]
[-36.95025  -8.98975 160.9455 ]
Merging: tile_54777 to tile_54795 

Small: tile_54792
[-36.213   -8.736  154.0997]
[-36.213   -8.736  154.0997]
Closest: tile_54791
[-36.13     -8.82175 153.9962 ]
[-36.13     -8.82175 153.9962 ]
Merging: tile_54792 to tile_54791 

Small: tile_54810
[-36.77225  -7.9875  157.9693 ]
[-36.7785   -7.99125 157.9458 ]
Closest: tile_54793
[-36.81275  -8.01875 157.971  ]
[-36.81275  -8.01875 157.971  ]
Merging: tile_54810 to tile_54793 

Small: tile_54811
[-36.01875  -7.19725 160.0345 ]
[-36.01875  -7.19725 160.0345 ]
Closest: tile_55017
[-35.99225  -7.24175 160.087  ]
[-35.99225  -7.24175 160.087  ]
Merging: tile_54811 to tile_55017 

Small: tile_54833
[-36.85125  -6.083   158.11   ]
[-36.99375  -6.20625 158.0835 ]
Closest: tile_54832
[-36.86875  -6.053   158.0278 ]
[-36.987    -6.18925 158.039  ]
Merging: tile_54833 to tile_54832 

Small: tile_54834
[-36.0775   -6.96675 159.919  ]
[-36.0775   -6.96675 159.919

In [56]:
with h5py.File(h5_small_full, 'r+') as h5f:
    keys = h5f.keys()
    print("Before Total Tiles:", str(len(keys)), "\n")
    deleted_keys = []
    for key in keys:
        if h5f[key]["coords"].shape[0] < 5:
            print("Small:", key)
            coords = h5f[key]["coords"][:]
            feats = h5f[key]["feats"][:]
            labels = h5f[key]["labels"][:]
            xyz_max = np.array([np.max(coords[:,0]), np.max(coords[:,1]), np.max(coords[:,2])])
            xyz_min = np.array([np.min(coords[:,0]), np.min(coords[:,1]), np.min(coords[:,2])])
#             print(xyz_max)
#             print(xyz_min)
            closest_tile = None
            closest_max = None 
            closest_min = None 
            for key_2 in keys:
                if key_2 == key or key in deleted_keys:
                    continue
                current_tile_data = h5f[key_2]["coords"][:]
                close_max = current_tile_data[KDTree(current_tile_data).query(xyz_max)[1]]
                close_min = current_tile_data[KDTree(current_tile_data).query(xyz_min)[1]]
                if closest_tile is None:
                    closest_tile = key_2
                    closest_max = close_max
                    closest_min = close_min
                else:
                    compare_max_arr = [closest_max, close_max]
                    temp_closest_max = compare_max_arr[KDTree(compare_max_arr).query(xyz_max)[1]]
                    compare_min_arr = [closest_min, close_min]
                    temp_closest_min = compare_min_arr[KDTree(compare_min_arr).query(xyz_max)[1]]
                    if not np.array_equal(temp_closest_max, closest_max) and not np.array_equal(temp_closest_min, closest_min):
                        closest_tile = key_2
                        closest_max = temp_closest_max
                        closest_min = temp_closest_min
            print("Closest:", closest_tile)
#             print(closest_max)
#             print(closest_min)
            print("Merging:", key, "to", closest_tile, "\n")
            np.concatenate((h5f[closest_tile]["coords"][:], coords))
            np.concatenate((h5f[closest_tile]["feats"][:], feats))
            np.concatenate((h5f[closest_tile]["labels"][:], labels))
            del h5f[key]
            deleted_keys.append(key)
    print("After Total Tiles:", str(len(h5f.keys())))

Before Total Tiles: 56250 

Small: tile_10004
Closest: tile_10024
Merging: tile_10004 to tile_10024 

Small: tile_10021
Closest: tile_10264
Merging: tile_10021 to tile_10264 

Small: tile_10092
Closest: tile_10118
Merging: tile_10092 to tile_10118 

Small: tile_10158
Closest: tile_10133
Merging: tile_10158 to tile_10133 

Small: tile_10191
Closest: tile_10215
Merging: tile_10191 to tile_10215 

Small: tile_10193
Closest: tile_10192
Merging: tile_10193 to tile_10192 

Small: tile_10199
Closest: tile_10200
Merging: tile_10199 to tile_10200 

Small: tile_1022
Closest: tile_1049
Merging: tile_1022 to tile_1049 

Small: tile_10341
Closest: tile_10340
Merging: tile_10341 to tile_10340 

Small: tile_1050
Closest: tile_786
Merging: tile_1050 to tile_786 

Small: tile_10514
Closest: tile_10513
Merging: tile_10514 to tile_10513 

Small: tile_10543
Closest: tile_10520
Merging: tile_10543 to tile_10520 

Small: tile_10544
Closest: tile_10523
Merging: tile_10544 to tile_10523 

Small: tile_10583
Cl

KeyboardInterrupt: 

In [31]:
with h5py.File(h5_small_full, 'r') as h5f:
    num_tiles = len(h5f.keys())
    num_train_tiles = int(num_tiles * 0.8)
    num_test_tiles = int(num_tiles * 0.1)
    num_validation_tiles = int(num_tiles * 0.1)
    print("Train Number of Tiles:", str(num_train_tiles))
    print("Test Number of Tiles:", num_test_tiles)
    print("Validation Number of Tiles:", num_validation_tiles)

Train Number of Tiles: 43521
Test Number of Tiles: 5440
Validation Number of Tiles: 5440


In [32]:
# Shuffle the tiles before splitting them into train/test/validate sets
with h5py.File(h5_small_full, 'r') as h5f:
    tile_names = list(h5f.keys())
    random.shuffle(tile_names)

In [33]:
with h5py.File(h5_small_full, 'r') as data:
    # Write train h5py
    with h5py.File(h5_small_train, 'w') as train:
        for key in tile_names[:num_train_tiles]:
            data.copy(data[key], train)
    # Write test h5py
    with h5py.File(h5_small_test, 'w') as test:
        for key in tile_names[num_train_tiles:num_train_tiles + num_test_tiles]:
            data.copy(data[key], test)
    # Write validate h5py
    with h5py.File(h5_small_validate, 'w') as validation:
        for key in tile_names[num_train_tiles + num_test_tiles:]:
            data.copy(data[key], validation)

In [30]:
# Check if it contains the correct data
with h5py.File(h5_small_full, 'r') as train:
    tile_names = list(train.keys())
    for tile in tile_names:
        if train[tile]["coords"].shape[0] == 1:
            print(tile)

In [36]:
# Check if it contains the correct data
with h5py.File(h5_small_full, 'r') as train:
    tile_names = list(train.keys())
    for tile in tile_names:
        print(train[tile]["coords"].shape[0])

11464
3895
5610
17
10248
54
10
577
244
2224
7043
7062
6562
6551
12145
6630
3769
2213
3100
1547
3283
2100
1250
1707
1839
7295
37
545
801
9
2576
3450
582
2840
3091
11996
1224
3057
3371
6732
7214
1469
3388
3332
3477
2332
9517
336
6
21
136
2393
4756
427
1647
1663
3215
2926
721
665
551
1417
4081
6109
3357
2478
1426
589
1511
459
96
534
17
423
40
14
2255
10445
4643
2226
3255
2110
2071
2745
2430
2574
396
864
3602
2147
1499
2912
900
1593
849
1469
2127
497
1387
298
90
2279
22
10
4633
2519
5233
7610
4128
2748
1376
18
1120
275
993
1698
1960
4659
4233
1994
1687
1894
2499
321
2457
1004
643
549
213
77
22
23
193
1157
272
10156
8104
8939
6563
7461
6768
6309
5223
3956
6964
310
6465
3059
3132
2832
4625
3387
1936
1962
615
786
323
132
140
217
53
195
2456
2354
5258
4645
282
257
4824
5495
467
2624
3426
6606
2771
172
5342
170
2283
942
1191
1164
301
96
111
5
2107
6440
129
7246
11793
7854
4496
4995
4860
4922
3390
3608
3134
207
1894
1992
2290
7901
1546
563
1159
646
700
365
187
14
7
3150
4150
5544
2680
950
48
49


780
3730
1349
93
4658
21
655
329
78
770
359
1064
682
129
299
415
246
564
637
320
175
4394
212
406
276
7683
3917
221
6830
6020
5838
1293
2146
4754
1167
170
828
190
1547
569
1073
192
2595
79
818
426
418
352
852
149
197
703
311
3168
2201
110
3191
1650
1299
3891
8314
8317
6704
3376
292
586
2944
3075
2378
1883
736
354
423
726
253
618
72
816
882
1220
537
25
153
964
221
12
6409
305
4785
2910
6082
7939
6077
4470
2415
3078
1639
3049
335
1698
550
191
132
78
178
547
190
6
984
393
96
38
689
500
311
337
90
689
252
372
490
537
13
65
14
185
838
1583
242
155
396
187
358
6
2789
2281
3253
3724
5531
4068
2423
2210
313
2161
2503
1586
1688
806
799
717
803
167
13
172
435
464
264
34
7
1438
735
1572
666
510
2504
50
1301
737
312
51
550
684
2298
978
14
355
67
381
19
93
15
1164
2211
5600
5036
7080
37
2634
2166
877
1138
623
547
1940
1138
118
167
8043
870
279
98
106
60
19
2758
2478
1389
954
5705
47
1828
783
1938
204
488
950
652
401
649
4046
940
377
426
453
311
1452
286
224
10
91
3064
104
32
2081
769
352
1052
632
3

742
12
131
467
34
99
418
637
256
125
141
94
261
190
456
221
204
474
10
85
12
10
1864
599
5
852
1789
459
99
284
2147
1718
337
515
140
134
418
360
102
54
5
125
137
78
9
7
1374
274
122
16
1136
1150
15
735
463
65
130
65
149
967
274
295
42
163
462
46
2473
1958
2905
2637
3624
3956
3865
3375
602
5
59
32
32
3664
8327
69
38
13
35
34
52
2138
1301
1077
1111
1141
1014
2314
624
635
800
332
1532
1825
486
268
775
110
927
465
199
292
3589
5339
171
978
29
744
187
145
297
215
102
22
59
1746
1208
79
175
526
6208
7430
1431
8
490
27
445
27
192
261
46
20
2138
331
40
270
1156
3051
960
3138
599
13
43
50
5
38
96
1863
331
208
134
71
10
3225
1091
1135
1631
2131
7175
1580
4335
4975
432
243
2128
91
615
634
531
455
38
115
109
88
124
2508
736
54
261
2702
5531
3543
1490
1635
148
469
838
1563
142
26
471
810
4170
972
225
38
65
64
65
10
2818
2992
1262
429
2036
3550
4528
3324
4832
3188
787
394
1429
767
1706
13
14
721
217
1001
243
71
120
116
7
598
27
17
2272
3313
2029
2584
5091
3143
2635
3314
239
5765
680
59
1318
482
423


8321
2262
1893
1410
3908
5375
5676
2982
1408
7963
99
107
9630
7363
1510
1223
2830
6954
1657
455
426
3241
56
2547
5171
2533
3094
1645
252
7480
10855
287
1737
7492
13
7581
2731
9408
90
5158
9556
190
4266
345
1752
852
115
8414
8681
13527
427
406
4948
8322
6507
911
1821
8989
7414
251
7473
6738
5421
6492
5
1202
1436
1204
2563
7910
9910
4858
4468
1118
4985
3261
262
10749
5026
2242
1172
9074
4059
2004
12753
6424
400
985
1913
842
904
877
105
10518
6113
12676
10263
3299
4032
4401
9966
3746
8807
8921
10820
1002
2016
882
113
609
678
813
555
1267
540
742
1712
638
6
7880
5462
8903
2221
12159
7746
15701
11995
185
8310
580
19
183
993
2159
2243
5065
2644
336
282
157
138
9341
7519
86
4399
7011
9840
17300
11248
315
12879
7512
3006
5
1267
5146
1866
3255
1953
1180
166
652
378
29
7057
3770
2080
144
3813
919
3757
39
991
1585
5640
1246
1799
2711
6154
6321
4246
3030
11
1505
1041
1032
328
17
18
8051
7252
3658
2602
53
1497
1765
2239
3722
6244
7370
3696
3498
1814
4579
45
4276
7339
3547
2661
1160
730
156
8650
912

191
247
5379
1215
785
1941
7745
3889
1087
2658
2422
1930
6761
3929
1911
367
37
245
15
32
90
1176
5073
211
727
175
8100
6238
8112
7364
4834
12465
5354
4217
7675
10955
8577
8723
9053
1529
82
135
157
137
3175
56
519
718
484
160
3160
3453
4356
4720
7238
2266
3389
4401
1760
2872
555
6949
5670
940
87
124
6500
242
490
172
57
12
448
113
45
27
4675
612
4808
3177
346
6612
2095
2126
104
2069
721
3554
6878
65
62
2713
189
297
1553
571
523
323
340
4771
187
124
181
43
3298
6620
1289
1363
2281
1951
1092
2217
1444
4359
2791
1186
4326
346
414
2518
3376
408
289
169
193
6
17
10
97
98
40
2936
8
5743
2666
313
316
240
1670
887
926
191
223
1073
419
493
871
1482
1599
1084
512
64
119
36
39
37
130
30
22
6
13370
3306
15
410
97
4140
7780
4980
3963
5448
5053
897
3202
5324
2492
1147
709
7386
858
973
91
291
5790
3364
3216
76
131
90
224
4911
3754
3572
5
71
605
355
24
5862
4862
6061
6020
2087
154
46
17
3916
3975
65
4749
859
4975
1643
1950
172
37
221
49
6905
7940
3182
1334
2944
1086
1013
2910
2225
5779
120
213
1178
4150

KeyboardInterrupt: 